Use this utlity to update the returns and std_dev fields within investment-options.csv

In [23]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [24]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [25]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [26]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [27]:
# Read in sp400.csv
sp400 = pd.read_csv('sp400.csv')
sp400.drop(columns=['SEC filings'], inplace=True)
sp400.rename(columns={'Ticker symbol': 'Symbol',
                      'Security':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp400.set_index("Symbol", inplace=True)
sp400

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AA,Alcoa,Materials,Aluminum,"Pittsburgh, Pennsylvania"
ACHC,Acadia Healthcare,Health Care,Health Care Facilities,"Franklin, Tennessee"
ACIW,ACI Worldwide,Information Technology,Application Software,"Miami, Florida"
ACM,AECOM,Industrials,Construction & Engineering,"Dallas, Texas"
ADNT,Adient,Consumer Discretionary,Auto Parts & Equipment,"Plymouth, Michigan"
...,...,...,...,...
X,U.S. Steel,Materials,Steel,"Pittsburgh, Pennsylvania"
XPO,XPO Logistics,Industrials,Trucking,"Greenwich, Connecticut"
XRX,Xerox,Information Technology,"Technology Hardware, Storage & Peripherals","Norwalk, Connecticut"


In [28]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [29]:
# Map sp400 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp400['Asset Class'] = sp400.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp400.index = sp400.index.str.replace('.', '-', regex=False)
sp400

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AA,Alcoa,US Stocks:Materials,Aluminum,"Pittsburgh, Pennsylvania"
ACHC,Acadia Healthcare,US Stocks:Healthcare,Health Care Facilities,"Franklin, Tennessee"
ACIW,ACI Worldwide,US Stocks:Technology,Application Software,"Miami, Florida"
ACM,AECOM,US Stocks:Industrials,Construction & Engineering,"Dallas, Texas"
ADNT,Adient,US Stocks:Consumer Discretionary,Auto Parts & Equipment,"Plymouth, Michigan"
...,...,...,...,...
X,U.S. Steel,US Stocks:Materials,Steel,"Pittsburgh, Pennsylvania"
XPO,XPO Logistics,US Stocks:Industrials,Trucking,"Greenwich, Connecticut"
XRX,Xerox,US Stocks:Technology,"Technology Hardware, Storage & Peripherals","Norwalk, Connecticut"


In [30]:
# Drop invalid symbols.
#sp400.drop(['AAXN'], inplace=True)

In [31]:
# Make symbols list.
symbols = list(sp400.index)
#symbols

In [32]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

AA ACHC ACIW ACM ADNT AFG AGCO AIRC ALE ALGM AM AMED AMG AMKR AN AR ARW ARWR ASB ASGN ASH ATR AVNT AVT AXON AYI AZPN AZTA BC BCO BDC BHF BJ BKH BLD BLDR BLKB BOH BRBR BRKR BRX BYD CABO CACI CADE CALX CAR CASY CATY CBRL CBSH CBT CC CELH CFR CGNX CHDN CHE CHH CHX CIEN CLF CLH CMC CNO CNX CNXC COHR COKE COLM COTY CPRI CR CRI CROX CRUS CSL CUBE CUZ CVLT CW DAN DAR DCI DECK DEI DINO DKS DOC DT DTM DY EEFT EGP EHC EME ENOV ENR ENS ENV EPR ESAB ESNT ETRN EVR EWBC EXEL EXLS EXP FAF FCFS FBIN FCN FFIN FHI FHN FICO FIVE FL FLO FLR FLS FNB FOXF FR FULT FYBR G GATX GBCI GEF GGG GHC GME GMED GNTX GO GPS GT GTLS GXO HAE HALO HBI HE HELE HIW HOG HOMB HQY HR HRB HUBB HWC HXL IAA IART IBKR IBOC ICUI IDA INGR IPGP IRDM IRT ITT JAZZ JBGS JBL JBLU JEF JHG JLL JWN KBH KBR KD KEX KMPR KMT KNSL KNX KRC KRG KSS LAD LAMR LANC LEA LECO LEG LFUS LHCG LII LITE LIVN LNTH LNW LOPE LPX LSCC LSI LSTR M MAC MAN MANH MASI MAT MDU MEDP MIDD MKSI MMS MODG MP MPW MRCY MSA MSM MTDR MTG MTSI MTZ MUR MUSA NARI NATI NAVI NBIX

In [33]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df

,AA,ACHC,ACIW,ACM,ADNT,AFG,AGCO,AIRC,ALE,ALGM,...,WTRG,WTS,WU,WWD,WWE,X,XPO,XRX,YETI,ZD
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,36.89,59.98,19.71,30.03,nan,34.26,39.40,nan,41.65,nan,...,22.24,58.05,13.02,45.96,10.70,24.82,14.05,26.58,nan,48.78
2015-01-05,34.75,59.12,19.26,28.72,nan,33.81,37.59,nan,40.99,nan,...,21.81,56.58,12.89,44.97,10.50,23.66,13.62,26.00,nan,47.99
2015-01-06,35.01,58.19,18.95,28.43,nan,33.62,37.42,nan,40.95,nan,...,21.84,55.51,12.87,44.65,10.53,22.94,13.14,25.66,nan,47.59
2015-01-07,35.91,60.63,19.03,29.01,nan,33.84,37.39,nan,41.72,nan,...,22.04,55.09,12.75,45.06,10.09,23.00,13.19,25.98,nan,47.78
2015-01-08,36.93,61.76,19.01,29.92,nan,34.46,38.19,nan,42.54,nan,...,22.18,55.66,13.00,44.84,9.98,23.50,13.26,26.70,nan,48.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-30,50.34,81.79,27.79,85.64,43.27,140.74,137.35,37.23,60.35,34.43,...,46.10,157.94,14.13,109.33,84.95,28.35,38.38,16.17,43.73,87.20
2023-01-31,52.24,84.02,27.93,87.27,45.02,142.59,138.13,38.26,61.86,38.17,...,46.73,163.52,14.17,102.26,84.62,28.49,39.86,16.38,44.76,89.48
2023-02-01,54.58,84.06,28.20,87.14,46.85,138.02,139.22,38.54,62.41,39.68,...,48.10,167.50,14.17,105.04,86.20,29.34,43.16,16.70,46.26,89.75


In [34]:
# Sample symbol.
symbol = 'ACHC'

In [35]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

51.32710109247225

In [36]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [37]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.25681432680453614

In [38]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.15980057607518972

In [39]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()

In [40]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     nan
2015-02-28    0.09
2015-03-31    0.13
2015-04-30   -0.04
2015-05-31    0.08
              ... 
2022-10-31    0.04
2022-11-30    0.10
2022-12-31   -0.08
2023-01-31    0.02
2023-02-28   -0.04
Freq: M, Name: ACHC, Length: 98, dtype: float64

In [41]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.43248301695481034

In [42]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [43]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp400.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [44]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')